In [1]:
import gc
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict
import tqdm.notebook as tqdm

In [10]:
aspect_abbr = 'cc'
train_df  = pd.read_csv('golabeler_train/'+aspect_abbr+'_df.csv')
train_df

,index,Entry,accessions,Sequence,Gene Ontology (cellular component),interpros,orgs
0,225,11K_PAVHV,P0DJZ0;,MQNNTTGMDTKSLKNCGQPKAVCTHCKHSPPCPQPGCVTKRPPVPP...,GO:0030430;GO:0033643;GO:0044216;GO:0044215;GO...,[],648237
1,252,13KDA_SCYCA,P83011;,MIFTAXDRSAIEXV,GO:0043227;GO:0005575;GO:0005622;GO:0044424;GO...,[],7830
2,253,13KDA_TRISC,P83010;,AGEPANNEDRFNY,GO:0043227;GO:0005575;GO:0005622;GO:0044424;GO...,[],30494
3,261,14310_ARATH,P48347; Q9LME5;,MENEREKQVYLAKLSEQTERYDEMVEAMKKVAQLDVELTVEERNLV...,GO:0005622;GO:0043231;GO:0043226;GO:0005829;GO...,"['IPR000308', 'IPR023409', 'IPR023410']",3702
4,266,14331_CAEEL,P41932; Q21537;,MSDTVEELVQRAKLAEQAERYDDMAAAMKKVTEQGQELSNEERNLL...,GO:0043227;GO:0005634;GO:0044464;GO:0005938;GO...,"['IPR000308', 'IPR023409', 'IPR023410']",6239
...,...,...,...,...,...,...,...
49341,550266,ZYX_CAEEL,Q9U3F4; H2L2F5; H2L2F6; H2L2F7; Q9U3F5;,MGPPPPPPPPPLLPSGEILPSRKWKTEDAPRRNNHPAPAPPKPSRP...,GO:0005634;GO:0030017;GO:0005622;GO:0044449;GO...,['IPR001781'],6239
49342,550267,ZYX_CHICK,Q04584;,MASPGTPGTRMTTTVSINISTPSFYNPQKKFAPVVAPKPKVNPFKT...,GO:0030055;GO:0005622;GO:0043226;GO:0005925;GO...,['IPR001781'],9031
49343,550268,ZYX_HUMAN,Q15942; A4D2G6; B4DQX7; Q6I9S4;,MAAPRPSPAISVSVSAPAFYAPQKKFGPVVAPKPKVNPFRPGDSEP...,GO:0005634;GO:0030055;GO:0005622;GO:0016021;GO...,['IPR001781'],9606
49344,550274,ZZZ3_HUMAN,Q8IYH5; B7WPC6; Q6N004; Q6N070; Q8IYP0; Q8IYR1...,MAASRSTRVTRSTVGLNGLDESFCGRTLRNRSIAHPEEISSNSQVR...,GO:0005634;GO:0031981;GO:0005622;GO:0043233;GO...,"['IPR009057', 'IPR017930', 'IPR001005', 'IPR00...",9606


In [11]:
file = open('golabeler_train/golabeler_train_'+aspect_abbr+'.fa','w')

for i in range(len(train_df)):
    file.write('>'+train_df.iloc[i]['Entry']+'\n'+train_df.iloc[i]['Sequence']+'\n')
file.close()

In [12]:
test_df  = pd.read_csv('golabeler_test/'+aspect_abbr+'_df.csv')
test_df

,level_0,index,Entry,accessions,Sequence,Gene Ontology (cellular component),interpros,orgs
0,84,617,1B48_HUMAN,P30486; Q29764;,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,GO:0044433;GO:0045335;GO:0031988;GO:0005794;GO...,"['IPR007110', 'IPR013783', 'IPR003006', 'IPR00...",9606
1,326,2607,8ODP_RAT,P53369;,MSTSRLYTLVLVLQPQRVLLGMKKRGFGAGRWNGFGGKVQEGETIE...,GO:0005634;GO:0031988;GO:0005622;GO:0043231;GO...,"['IPR020476', 'IPR020084', 'IPR000086', 'IPR01...",10116
2,392,3170,AA3R_MOUSE,Q61618; Q9R202;,MEADNTTETDWLNITYITMEAAIGLCAVVGNMLVIWVVKLNPTLRT...,GO:0005622;GO:0016021;GO:0000323;GO:0043231;GO...,"['IPR000466', 'IPR001634', 'IPR000276', 'IPR01...",10090
3,462,3586,AARE_ARATH,Q84LM4; O23313;,MDSSGTDSAKELHVGLDPTTEEEYATQSKLLQEFINIPSIDKAWIF...,GO:0043227;GO:0005634;GO:0005575;GO:0005622;GO...,"['IPR011042', 'IPR029058', 'IPR011659', 'IPR00...",3702
4,575,3984,ABA4_ARATH,Q8LFP9; Q9FZH4;,MGFSSFISQPLSSSLSVMKRNVSAKRSELCLDSSKIRLDHRWSFIG...,GO:0005622;GO:0043231;GO:0043226;GO:0044435;GO...,['IPR025461'],3702
...,...,...,...,...,...,...,...,...
1143,55732,552113,ZN809_MOUSE,G3X9G7; Q4KL58; Q8BIJ2;,MGLVSFEDVAVDFTLEEWQDLDAAQRTLYRDVMLETYSSLVFLDPC...,GO:0043227;GO:0005634;GO:0044464;GO:0005622;GO...,"['IPR001909', 'IPR007087', 'IPR015880', 'IPR01...",10090
1144,55751,552243,ZNF8_MOUSE,Q8BGV5; Q52KP6; Q8BJ50;,MDHQDKAATVAMASRPQATQLQEPVTFRDVAVDFTQEEWGQLDPTQ...,GO:0043227;GO:0005634;GO:0044464;GO:0005622;GO...,"['IPR001909', 'IPR007087', 'IPR015880', 'IPR01...",10090
1145,55763,552282,ZNRF4_HUMAN,Q8WWF5; A8K886; O75866;,MPLCRPEHLMPRASRVPVAASLPLSHAVIPTQLPSRPGHRPPGRPR...,GO:0005783;GO:0005622;GO:0043231;GO:0043226;GO...,"['IPR003137', 'IPR001841', 'IPR013083']",9606
1146,55764,552284,ZNRF4_MOUSE,Q9DAH2; Q9WTN2;,MARFAWTRVAPVALVTFWLVLSLSPTDAQVNLSSVDFLDLPALLGV...,GO:0005575;GO:0005622;GO:0044424;GO:0005623;GO...,"['IPR001841', 'IPR013083']",10090


In [13]:
file = open('golabeler_test/golabeler_test_'+aspect_abbr+'.fa','w')

for i in range(len(test_df)):
    file.write('>'+test_df.iloc[i]['Entry']+'\n'+test_df.iloc[i]['Sequence']+'\n')
file.close()